In [2]:
import numpy as np
import pandas as pd

from backtesting import Strategy
from backtesting.lib import crossover

from backtesting import Backtest

from scripts.data_preparation import load_trades_from_csv, get_bar_stats

In [3]:
path = 'data/trades_RIU2@FORTS_2022_08_19_2022_08_19.csv'
trades = load_trades_from_csv(path)
# display(trades)
# trades.info()
# display('Duplicates count',trades.duplicated().sum())

In [4]:
resampled = trades.set_index('datetime').groupby(pd.Grouper(freq='1Min'))
# time_bars = get_bar_stats(resampled)
one_minute_bar = get_bar_stats(resampled)
one_minute_bar = one_minute_bar.dropna()

In [5]:
one_minute_bar

,open,high,low,close,vwap,volume,txn
datetime,,,,,,,
2022-08-19 10:00:00,111670.0,111970.0,111300.0,111700.0,0,1455,832
2022-08-19 10:01:00,111700.0,111850.0,111580.0,111670.0,0,731,466
2022-08-19 10:02:00,111670.0,111670.0,111400.0,111520.0,0,715,413
2022-08-19 10:03:00,111530.0,111730.0,111360.0,111430.0,0,636,440
2022-08-19 10:04:00,111450.0,111670.0,111210.0,111250.0,0,1014,563
...,...,...,...,...,...,...,...
2022-08-19 23:45:00,112480.0,112480.0,112450.0,112470.0,0,23,12
2022-08-19 23:46:00,112470.0,112490.0,112450.0,112490.0,0,31,14
2022-08-19 23:47:00,112490.0,112490.0,112430.0,112440.0,0,32,7


Is 1 minute grouping correct? Is minute 10:00:00 consists of all trades from 10:00:00 to 10:00:59.999? Is it the correct way to build bars?

In [ ]:
path = 'data/trades_RIU2@FORTS_2022_08_23_2022_08_23.csv'
trades_valid = load_trades_from_csv(path)
# display(trades_valid)
# trades_valid.info()
# display('Duplicates count',trades_valid.duplicated().sum())
resampled_valid = trades_valid.set_index('datetime').groupby(pd.Grouper(freq='1Min'))
one_minute_bar_valid = get_bar_stats(resampled_valid)
one_minute_bar_valid = one_minute_bar_valid.dropna()

In [ ]:
data = pd.concat([one_minute_bar,one_minute_bar_valid])
data.columns = ['Open','High','Low','Close','VWAP','Volume','Txn']

In [ ]:
data

In [ ]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [ ]:
class SmaCross(Strategy):
    n1 = 10
    n2 = 20
    
    def init(self):
        self.sma1 = self.I(SMA, self.data['Close'], self.n1)
        self.sma2 = self.I(SMA, self.data['Close'], self.n2)
        
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1,self.sma2):
            self.position.close()
            self.buy()
        
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [ ]:
bt = Backtest(data, SmaCross, cash=10_000_000, commission=.002)
stats = bt.run()
stats

In [ ]:
bt.plot()

In [ ]:
%%time

stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

In [ ]:
stats._strategy

In [ ]:
bt.plot(plot_volume=False, plot_pl=False)

In [ ]:
stats.tail()

In [ ]:
stats['_equity_curve']  # Contains equity/drawdown curves. DrawdownDuration is only defined at ends of DD periods.

In [ ]:
stats['_trades']  # Contains individual trade data